In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is

In [4]:
# from ml.preprocess import data
import pickle
from dask_ml.preprocessing import StandardScaler
from dask_ml.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score

import numpy as np
import pandas as pd

from utils import helper, config, dasker

from ml.models.base.slug_ann import SlugANN
from ml.models.base.slug_xgboost import SlugXGBoost
from ml.models.base.brisk_xgboost import BriskXGBoost
from ml.models.ensemble import BaseUnify


In [5]:
client = dasker.get_local_client()
# cl =  dasker.get_global_client()
print(client)

2022-10-26 12:46:49,241 - asyncio - DEBUG - Using selector: EpollSelector
<Client: 'tcp://127.0.0.1:8786' processes=4 threads=4, memory=15.37 GiB>


In [6]:
df_X, df_y = helper.get_covid_dataset()
df_X = df_X.drop(['location'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.33, random_state=config.rand_state)

X_scalar = StandardScaler().fit_transform(df_X.copy())

X_base_scalar, X_test_scalar, y_base_scalar, y_test_scalar = train_test_split(X_scalar, df_y, test_size=0.33, random_state=config.rand_state)
X_train_scalar, X_val_scalar, y_train_scalar, y_val_scalar = train_test_split(X_base_scalar, y_base_scalar, random_state=config.rand_state)


In [7]:
### BaseClass
slug_ann = SlugANN()
slug_ann.pred_class = 'regression'
slug_ann.X_train = X_train_scalar
slug_ann.X_test = X_test_scalar
slug_ann.X_val = X_val_scalar
slug_ann.y_train = y_train_scalar
slug_ann.y_test = y_test_scalar
slug_ann.y_val = y_val_scalar
slug_ann.n_trials = 20

### BaseClass
slug_xgb = SlugXGBoost()
slug_xgb.pred_class = 'regression'
slug_xgb.objective = 'count:poisson'
slug_xgb.X_train = X_train
slug_xgb.X_test = X_test
slug_xgb.y_train = y_train
slug_xgb.y_test = y_test
slug_xgb.n_trials = 20

base_models = [slug_ann, slug_xgb]

In [13]:
config.create_project_dirs(overwrite=False)

Directory structure for project /covid created successfully


In [8]:
def __timer_expired__(sig, frame):
    print('\n')
    # print(f'timer of {timer}s expired')    
    print('Ensemble training session interrupted due to timer runout')
    
    client.shutdown()
    unified_models.fetch_models(retrain=False)    

In [9]:
unified_models = BaseUnify(timer=20, base_models=base_models, callback_timer_expired=None)
unified_models.ensemble_n_trials = 200


In [10]:
unified_models.fetch_models(retrain=False)

In [11]:
unified_models.fit_ensemble(retrain=False)

2022-10-26 12:47:03,763 - console_info - INFO - Best score from base models: -3.829694062577821 compared to ensemble score:-3.0096565489768223
2022-10-26 12:47:03,763 - console_info - INFO - Best score from base models: -3.829694062577821 compared to ensemble score:-3.0096565489768223
2022-10-26 12:47:03,765 - console_info - INFO - Ensemble selected
2022-10-26 12:47:03,765 - console_info - INFO - Ensemble selected


In [19]:
unified_models.best_base_model

In [12]:
pred = unified_models.predict(df_X)

In [13]:
r2_score(pred, df_y)

-0.07466123508229416

2022-10-26 13:21:27,958 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-10-26 13:21:27,959 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-10-26 13:21:27,959 - distributed.nanny - ERROR - Worker process died unexpectedly
2022-10-26 13:21:27,974 - distributed.nanny - WARNING - Restarting worker
2022-10-26 13:21:27,975 - distributed.nanny - WARNING - Restarting worker
2022-10-26 13:21:27,985 - distributed.nanny - WARNING - Restarting worker
2022-10-26 13:21:28,008 - distributed.nanny - WARNING - Restarting worker
2022-10-26 13:21:28,011 - distributed.nanny - WARNING - Restarting worker
2022-10-26 13:21:28,038 - distributed.nanny - WARNING - Restarting worker
2022-10-26 13:21:28,042 - distributed.nanny - WARNING - Restarting worker
2022-10-26 13:21:28,063 - distributed.nanny - WARNING - Restarting worker
2022-10-26 13:21:28,065 - distributed.nanny - WARNING - Restarting worker
2022-10-26 13:21:28,141 - distributed.nanny - WARNING - Restarting worker

In [ ]:
pred = unified_models.predict(X_train)
r2_score(pred, y_train)

In [ ]:
pred = unified_models.predict(X_test)
r2_score(pred, y_test)